In [1]:
import os

In [2]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject/research'

In [3]:
%cd EndToEndRedWineQualityMLProject

[Errno 2] No such file or directory: 'EndToEndRedWineQualityMLProject'
/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject/research


In [4]:
os.chdir('../')

In [5]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject'

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    scaler_path: Path
    metrics_file_name: Path
    target_column: str


In [7]:
from RedWineQualityMLProject.constants import *
from RedWineQualityMLProject.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(schema_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(self.config.artifacts_root)
    def get_modelEvaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        create_directories(config.root_dir)
        return ModelEvaluationConfig(root_dir=config.root_dir, test_data_path=config.test_data_path, model_path=config.model_path,scaler_path=config.scaler_path,metrics_file_name=config.metrics_file_name,target_column=schema.name )
    

In [8]:
import pandas as pd
import numpy as np
import pickle as pkl
from RedWineQualityMLProject.utils.common import save_json
from RedWineQualityMLProject import logger
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [9]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    def calc_metrics(self,y_test,y_test_pred):
        r2 = r2_score(y_test, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        return r2, mae, rmse
    def evaluate(self):
        with open(self.config.model_path,'rb') as f:
            model = pkl.load(f)
        with open(self.config.scaler_path,'rb') as f:
            scaler = pkl.load(f)
        test_data = pd.read_csv(self.config.test_data_path)
        x_test = test_data.drop(self.config.target_column, axis = 1)
        y_test = test_data[self.config.target_column]
        x_test = scaler.transform(x_test)
        y_test_pred = model.predict(x_test)
        r2, mae, rmse = self.calc_metrics(y_test,y_test_pred)
        logger.info('Model Evaluation Completed Successfully')
        logger.info(f'Metrics are r2 = {r2}, mae={mae}, rmse={rmse}')
        scores = {'r2':r2,'mae': mae, 'mse': rmse}
        save_json(path= Path(self.config.metrics_file_name), data=scores) 


        




In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_modelEvaluation_config()
    model_evalution = ModelEvaluation(config=model_evaluation_config)
    model_evalution.evaluate()
except Exception as e:
    raise e


[2023-10-01 15:14:27,263: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-01 15:14:27,301: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 15:14:27,337: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-01 15:14:27,346: INFO: common: created directory at: artifacts]
[2023-10-01 15:14:27,353: INFO: common: created directory at: artifacts/model_evaluation]
[2023-10-01 15:14:27,583: INFO: 2507249269: Model Evaluation Completed Successfully]
[2023-10-01 15:14:27,594: INFO: 2507249269: Metrics are r2 = 0.48761283152311186, mae=0.42245000000000005, rmse=0.5629915629918445]
[2023-10-01 15:14:27,603: INFO: common: json file saved at : artifacts/model_evaluation/metrics.json]
